# **IDL Assignment 06**

Team Members:

Libin Kutty - Group 2

Ritu Gahir - Group 3

Viju Sudhi - Group 10

In [2]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding, SimpleRNN
from keras import Model, Input
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
!python prepare_data2.py -m 500 shakespeare_input.txt regex '\n\n+'

2020-06-02 11:23:38.995646: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 31022 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
29429 sequences remaining.
Longest remaining sequence has length 499.
Removing length-0 sequences...
29429 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences..

In [4]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("regex.tfrecords")


# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))
print(data)

# a map from characters to indices
vocab = pickle.load(open("regex_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

<MapDataset shapes: (None,), types: tf.int32>
{'z': 3, 'W': 4, 'B': 5, '?': 6, 'f': 7, 'q': 8, '\n': 9, ';': 10, 'U': 11, 'Q': 12, 'P': 13, 's': 14, 'e': 15, 'm': 16, 'S': 17, 'b': 18, 'w': 19, "'": 20, 'O': 21, 'd': 22, 'V': 23, 'n': 24, 'H': 25, 'o': 26, '.': 27, 'y': 28, 'E': 29, '&': 30, 'i': 31, 'u': 32, 'C': 33, '!': 34, 'p': 35, 'Z': 36, '[': 37, 'L': 38, 'F': 39, 'N': 40, '$': 41, 'r': 42, ' ': 43, 'k': 44, 'v': 45, 'h': 46, 'R': 47, 'X': 48, ':': 49, 'a': 50, ',': 51, 'x': 52, 'I': 53, 'T': 54, 'j': 55, 'J': 56, 'K': 57, '-': 58, 'G': 59, '3': 60, 'c': 61, ']': 62, 'M': 63, 't': 64, 'Y': 65, 'A': 66, 'g': 67, 'l': 68, 'D': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70


In [0]:
def getChar(inp):
  s = [] 
  for ind in inp.numpy():
    s.append(ind)
  
  txt = ''.join(ind_to_ch[ix] for ix in s)
  print(txt) 

In [6]:
# view data
count = 0
s = []
for i in data:
  count+=1
  if count == 10:
    print(i)
    getChar(i)

txt = ''.join(ind_to_ch[ix] for ix in s)  
print(txt)
print(count)

tf.Tensor(
[ 1 44 61 66 16 42 11 14 58  7  8  7 25 61 42 27 39  9 16 57 37 11 14 20
 16 57 14 18 68 16 66 61 61 11 14 61 19 18 61 66  7 13 37 37 20 14 13 45
 13  7 42 19  8 14 58 13  7 57 19 14 52 13 68 66  7 57 19 30  2], shape=(69,), dtype=int32)
<S>Second Citizen:
Would you proceed especially against Caius Marcius?</S>

29429


In [0]:
BATCH_SIZE = 128

In [7]:
data_ = data.padded_batch(BATCH_SIZE)
count = 1
s = []
for ind in data_:
  count_1 = 1
  for i in ind:
    count+=1
    count_1+=1
    if count == 13:
      print(i)
      getChar(i)
print("Number of Batches", count)
print("Number of el in a batch", count_1)

txt = ''.join(ind_to_ch[ix] for ix in s)
print(txt)

tf.Tensor(
[ 1 62 57  9 14 30 56 61 12 63 52 63 66 57 30 65  3 12 14 30  4 63 56 57
 31 61 27 14  6 61 41 34 44 52 61  4 57 31 28 63  9 57  4 61 34 57 61 34
 44  4 61 56 14 30 57 61 39 14 31 61 34 63  4 61  9 14  6 30 52 31 27 26
  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [0]:
BATCH_SIZE = 128

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, 499]),
    tf.keras.layers.SimpleRNN(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim = embedding_dim,
  rnn_units = rnn_units,
  batch_size = BATCH_SIZE)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, 499, 128)           8960      
_________________________________________________________________
simple_rnn (SimpleRNN)       (128, 499, 1024)          1180672   
_________________________________________________________________
dense (Dense)                (128, 499, 70)            71750     
Total params: 1,261,382
Trainable params: 1,261,382
Non-trainable params: 0
_________________________________________________________________


In [0]:
def split_input_target(chunk):
    input_text = chunk[1:][:-1]
    target_text = chunk[1:][1:]
    return input_text, target_text

dataset = data.map(split_input_target)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
dataset= dataset.shuffle(46000).repeat()
dataset=dataset.padded_batch(128, padded_shapes=([499],[499]),drop_remainder=True)

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=target, logits=predictions, name=None)
    zero_count = tf.math.count_nonzero(inp, axis=1, keepdims=None, dtype=tf.dtypes.int64, name=None)-1
    mask = tf.sequence_mask(zero_count, 499,dtype=tf.dtypes.float32)
    masked_loss = tf.math.multiply(loss,mask)
    red_masked_loss = tf.cast(tf.math.reduce_sum(masked_loss, axis=[0, 1], keepdims=False, name=None),tf.float32)
    total_zero_count = tf.cast(tf.reduce_sum(zero_count,0),tf.float32)
    final_loss = red_masked_loss/total_zero_count
    
  grads = tape.gradient(final_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return final_loss

In [0]:
# Training step
EPOCHS = 5

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)
    if batch_n>1000:
      break 

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.218281269073486
Epoch 1 Batch 100 Loss 3.362417221069336


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
seq_len=1
model1 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=seq_len)

model1.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model1.build(tf.TensorShape([1, None]))
model1.summary()

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))